<a href="https://colab.research.google.com/github/d-bowen/PowerBI-Workshop/blob/master/NAS_2020_Opinionated_Guide_to_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://bit.ly/35TT8xG

# Why an Opionated Guide to Pandas?
* “There should be one—and preferably only one—obvious way to do it” — Zen of Python.
* There are many, many ways to do the same thing in pandas. 
* It is hard to tell if they do the exact same thing or which one you should use.
* That's why I made An Opinionated Guide to pandas—to present you one consistent (and a bit opinionated) way of doing data analysis & science with pandas.

# What is Data Analysis?
* "Data analysis is a process of inspecting, cleansing, transforming and modeling data with the goal of discovering useful information, informing conclusions and supporting decision-making," from the source of all truth—[Wikipedia](https://en.wikipedia.org/wiki/Data_analysis)

# What tools are available?
* Excel / Tableau / etc.
    * Expensive
    * Limited
    * Easy to learn
    * Supported
* Programming language
    * Free
    * Incredibly powerful
    * Steep learning curve
    * pandas can read from and write to Excel files
    * Data Analysts that know Python & SQL make more money!

## Why Python/pandas?
* Relatively simple to learn
* pandas DataFrames are similar data structures to Excel
* Many powerful libraries
* Fantastic community, documentation, conferences, & best of all [free tutorials](https://www.youtube.com/channel/UCQTQ0AbOupKNxKKY-_x46OQ)

## Why Jupyter Notebooks
* Interactive environment
* Less analyst/scientist visual emphasis on the data (unlike say Excel or Tableau), more on the results you are after
* Python (as well as many other language) interpreter 
    * The name Jupyter is derived from Julia, Python, R
* With markdown Jupyter Notebooks provide fantastic (internal) reporting capability
* User friendly

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Intro to Data Structures

First off, a lot of the materials presented here come from the pandas [Intro to Data Structures Tutorial](https://pandas.pydata.org/pandas-docs/stable/dsintro.html). That being said I hope to add some color on what is important to learn from pandas by telling you the idioms that I use on a day to day basis as a data scientist. So let's start off.

## Series

Pandas has two data structures a Series and a Dataframe. A series is like a column in excel, basically a list of datapoints all of the same type. And the basic way to create a series object is below:

In [ ]:
pd.Series?

In [ ]:
s = pd.Series(
        np.random.randn(5), 
        index=['a', 'b', 'c', 'd', 'e'], 
        name='example')

s

a   -0.950332
b    0.069464
c    0.931217
d    0.257873
e    1.575623
Name: example, dtype: float64

There are other ways to make a series (like from a dictionary), but in general this is the only one that I ever use. So notice that a series has basically three important parts:

1. The data
2. The index 
3. The name

The data can be a list of data, or a single instance that broadcasts, like below:

In [ ]:
pd.Series(5, index=['a', 'b', 'c', 'd', 'e'])

a    5
b    5
c    5
d    5
e    5
dtype: int64

(Broadcasting as we will see later on is really important).

As a data analyst or scientist the data is what you are primarily interested in.
* The index is often used in time series, but otherwise I really don't use the index for series (now I do use the index for dataframes quite a lot!). 
* But notice that each datapoint is associated with an index.
* Finally the name. The name is only really important when you add a series to a dataframe. In that case the name of the series becomes the column. 

So far you have not seen why series are all that useful, but now we start to get into it. Series have various ways that you can index into them:

In [ ]:
s[0]

-0.9503317519332788

In [ ]:
s[:3]

a   -0.950332
b    0.069464
c    0.931217
Name: example, dtype: float64

In [ ]:
s[[4, 3, 1]]

e    1.575623
d    0.257873
b    0.069464
Name: example, dtype: float64

In [ ]:
s.values

array([-0.95033175,  0.06946445,  0.93121664,  0.25787339,  1.57562333])

In [ ]:
s['e'] = 500
s

a     -0.950332
b      0.069464
c      0.931217
d      0.257873
e    500.000000
Name: example, dtype: float64

Generally Speaking I don't do any of the above operations - and if you find yourself using them definitely give some thought on whether you should be using pandas for those operations or whether you should be using Numpy.

Now let me show you some operations that I frequently use:

In [ ]:
s[[True, True, False, False, True]]

a     -0.950332
b      0.069464
e    500.000000
Name: example, dtype: float64

In [ ]:
# or the extremely common
s[s > 0], s > 0

(b      0.069464
 c      0.931217
 d      0.257873
 e    500.000000
 Name: example, dtype: float64, a    False
 b     True
 c     True
 d     True
 e     True
 Name: example, dtype: bool)

In [ ]:
# and you can mutate the data too
# you'll just need to be careful with this!
s[s < 0] *= -1

In [ ]:
s

a      0.950332
b      0.069464
c      0.931217
d      0.257873
e    500.000000
Name: example, dtype: float64

But one thing that is super useful about series is that you can do vectorized operations (fast computations on everything in the entire series) on them. And you have already seen one. 

In [ ]:
s > 0

a    True
b    True
c    True
d    True
e    True
Name: example, dtype: bool

In [ ]:
s + s

a       1.900664
b       0.138929
c       1.862433
d       0.515747
e    1000.000000
Name: example, dtype: float64

In [ ]:
%%timeit
s.mean()

The slowest run took 8.76 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 32.3 µs per loop


In [ ]:
# just be careful with some operations
# if the indexes don't match up you will get nans
s + s[s > 0]

a       1.900664
b       0.138929
c       1.862433
d       0.515747
e    1000.000000
Name: example, dtype: float64

These types of operations that are over columns is what pandas is made for. Any time you stray from doing operations over columns, you should think to yourself: is pandas the best tool for me?

Now doing operations over one column might seem useful, but what about operations over multiple columns.

### pandas Series Recap
* Kind of like Excel column
* Comprised of:
    * data
    * index
    * name
* Columnar operations are what pandas is best and fastest at
* Building blocks of DataFrames

### Questions?

## DataFrames

Series are nice, but the really nice thing about them is that you can build DataFrames with them. DataFrames are like an entire Excel spreadsheet! As you can probably guess, dataframes are a list of Series, each one with a name and the same index. Thus an easy way to create a dataframe is to create it with a dictionary of series/lists:

In [ ]:
d = {'one' : pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])}

df = pd.DataFrame(d)

df

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


In [ ]:
# often I actually just pass in np arrays and scalars

d = {'one' : 'Hellow',
    'two' : np.array([1., 2., 3., 4.])}

df = pd.DataFrame(d)
df

,one,two
0,Hellow,1.0
1,Hellow,2.0
2,Hellow,3.0
3,Hellow,4.0


There are plenty of ways to create one of these things, but generally speaking just knowing one is enough. You can always change the values of the index or the columns later:

In [ ]:
df.columns = ['1', '2']
df.index = ['a', 'b', 'c', 'd']
df

,1,2
a,Hellow,1.0
b,Hellow,2.0
c,Hellow,3.0
d,Hellow,4.0


DataFrames are basically just dictionaries of columns/Series.  So, you can use most of the same techniques you used for Series on DataFrames themselves. 

The general way to ref a column is below:

In [ ]:
d = {'one' : 'Hellow',
    'two' : np.array([1., 2., 3., 4.])}

df = pd.DataFrame(d)
df.index = ['a', 'b', 'c', 'd']

# gives you back a named series
df['one']

a    Hellow
b    Hellow
c    Hellow
d    Hellow
Name: one, dtype: object

You can then do anything with the series that we did above, nifty.

There is a dot notation shortcut, but it is almost better not to know it because it can lead to errors if not used correctly!

You can of course delete and make new columns, with broadcasting as well

In [ ]:
del df['one']

In [ ]:
df['three'] = df['two'] + df['two']
df['four'] = 'four'
df['five'] = df['four'][:2]

In [ ]:
df

,two,three,four,five
a,1.0,2.0,four,four
b,2.0,4.0,four,four
c,3.0,6.0,four,NaN
d,4.0,8.0,four,NaN


Again there are other ways of inserting columns (insert and assign methods) but I never use them. The benefits of using other methods also seems pretty small.

Next let's go over indexing and selecting with dataframes. There are basically 4 ways to do so:

In [ ]:
# get a column
df['two']

a    1.0
b    2.0
c    3.0
d    4.0
Name: two, dtype: float64

In [ ]:
# or more
df[['five', 'two']]

,five,two
a,four,1.0
b,four,2.0
c,NaN,3.0
d,NaN,4.0


In [ ]:
# select by indexes and column names
df.loc['a', 'two']

1.0

In [ ]:
df.loc['d':'a':-1, 'two':'three']

,two,three
d,4.0,8.0
c,3.0,6.0
b,2.0,4.0
a,1.0,2.0


In [ ]:
# select rows and columns by their ordering
df.iloc[1:3, 0]

b    2.0
c    3.0
Name: two, dtype: float64

In [ ]:
df.iloc[1:3]

,two,three,four,five
b,2.0,4.0,four,four
c,3.0,6.0,four,NaN


## DataFrame Functions

In addition to doing these columnwise operations, you can also do some DataFramewise operations. 

The most useful of these is the copy method, it makes a copy :)

In [ ]:
df.copy()

,two,three,four,five
a,1.0,2.0,four,four
b,2.0,4.0,four,four
c,3.0,6.0,four,NaN
d,4.0,8.0,four,NaN


The astype method converts the types of columns =

In [ ]:
df.two.astype(np.int)

a    1
b    2
c    3
d    4
Name: two, dtype: int64

The next thing that I very commonly use is the dataframe transpose ability:

In [ ]:
df.T

,a,b,c,d
two,1,2,3,4
three,2,4,6,8
four,four,four,four,four
five,four,four,NaN,NaN


This puts the rows as the columns and the columns as the rows. It can be a good way to do row-wise operations, but mainly I do it to display dataframe values. Below are the three common ways to display dataframe values:

In [ ]:
df.head(2)

,two,three,four,five
a,1.0,2.0,four,four
b,2.0,4.0,four,four


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   two     4 non-null      float64
 1   three   4 non-null      float64
 2   four    4 non-null      object 
 3   five    2 non-null      object 
dtypes: float64(2), object(2)
memory usage: 320.0+ bytes


In [ ]:
df.describe(include='all')

,two,three,four,five
count,4.000000,4.000000,4,2
unique,NaN,NaN,1,1
top,NaN,NaN,four,four
freq,NaN,NaN,4,2
mean,2.500000,5.000000,NaN,NaN
std,1.290994,2.581989,NaN,NaN
min,1.000000,2.000000,NaN,NaN
25%,1.750000,3.500000,NaN,NaN
50%,2.500000,5.000000,NaN,NaN
75%,3.250000,6.500000,NaN,NaN


You will notice however that when the number of columns is too much the display is messy:

In [ ]:
for i in range(20):
    df[i] = i
    
df.head()

,two,three,four,five,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
a,1.0,2.0,four,four,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
b,2.0,4.0,four,four,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
c,3.0,6.0,four,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
d,4.0,8.0,four,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19


In [ ]:
#transposing helps
df.head().T

,a,b,c,d
two,1,2,3,4
three,2,4,6,8
four,four,four,four,four
five,four,four,NaN,NaN
0,0,0,0,0
1,1,1,1,1
2,2,2,2,2
3,3,3,3,3
4,4,4,4,4
5,5,5,5,5


Sometimes this will also truncate. To view more you can always change the view options below (btw, there are many many options in pandas, you can check them all out either [here](https://pandas.pydata.org/pandas-docs/stable/options.html) or with a `pd.set_option?`):

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('precision', 7)

## Data Structures Conclusion

Congrats, that's the _basics_.  After this Nashville Analytics Summit workshop you should be able to do the [Getting and knowing Exercises](https://github.com/guipsamora/pandas_exercises#getting-and-knowing) here.

There is much more to do a know about pandas, we'll be going through more here!

## Questions?

# Exercises — Getting and Knowing your Data
* Source: [guipsamora]( https://github.com/guipsamora/pandas_exercises/blob/master/01_Getting_%26_Knowing_Your_Data/Chipotle/Exercises.ipynb)
* After the Nashville Analytics Summit, check out [Chipotle Exercises Video Tutorial](https://www.youtube.com/watch?v=lpuYZ5EUyS8&list=PLgJhDSE2ZLxaY_DigHeiIDC1cD09rXgJv&index=2) to watch a data scientist go through the exercises.

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [ ]:
# Already completed above
# import numpy as np
# import pandas as pd

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

In [ ]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
df = pd.read_csv(url, sep='\t')

### Step 3. Assign it to a variable called chipo.

### Step 4. See the first 10 entries

### Step 5. What is the number of observations in the dataset?

In [ ]:
# Solution 1


In [ ]:
# Solution 2


### Step 6. What is the number of columns in the dataset?

### Step 7. Print the name of all the columns.

### Step 8. How is the dataset indexed?

### Step 9. Which was the most-ordered item? 

### Step 10. For the most-ordered item, how many items were ordered?

# Pandas Indexing and Selecting

Let's talk about slicing and dicing pandas data. We are going to be going over four topics today:

* Review the basics
* Multi-index
* Getting Single Values
* Pointing out some stuff you don't need to worry about

As always you can check out the full documentation: [basic indexing](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) and [advanced indexing](http://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html). But be warned that they are very long and tell you way more than you'd need to know :)

## Review the Basics

First let's start with a bit of a recap on traditional indexing and selection. (We went over most of this in the [pandas fundamentals](https://github.com/knathanieltucker/pandas-tutorial/blob/master/notebooks/Pandas%20Intro%20to%20Data%20Structures.ipynb)). To start off with, here is the data we are going to be working with (good old tips data):

In [ ]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


There are basically 4 ways to do get data from dataframes:

In [ ]:
# 1) get columns
tips[['total_bill', 'tip']].head()

,total_bill,tip
0,16.99,1.01
1,10.34,1.66
2,21.01,3.50
3,23.68,3.31
4,24.59,3.61


In [ ]:
# 2) get some rows
tips[3:5]

,total_bill,tip,sex,smoker,day,time,size
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
# 3) select rows and columns based on their name
tips.loc[2:4, 'sex': 'smoker']

,sex,smoker
2,Male,No
3,Male,No
4,Female,No


In [ ]:
# select rows and columns by their ordering
tips.iloc[1:3, 0:2]

,total_bill,tip
1,10.34,1.66
2,21.01,3.50


In [ ]:
# 5) select using a bool series
tips[tips['tip'] > 1].head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


But this is just the tip of the iceberg (well actually it's 90% of the iceberg). 

But there are a couple of other important concepts that you will most likely get into when diving into other pandas functionalities.

# Multi-index

A subject that you might not think that you'd need - but turns out to be a rather frequent usecase. 

The initial idea behind the multi-index was to provide a framework to work with higher dim data (and thus a replacement for panels).

But because of some operations it became quite commonplace. In almost all cases multi-index comes from [groupby's](https://github.com/knathanieltucker/pandas-tutorial/blob/master/notebooks/Group%20Operations.ipynb) (you will almost never construct it or read it in yourself).

Let's do an example below:

In [ ]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
mi_tips = tips.groupby(['sex', 'smoker']).agg({'tip': 'mean'})
mi_tips

tip
sex    smoker           
Male   Yes     3.0511667
       No      3.1134021
Female Yes     2.9315152
       No      2.7735185

In [ ]:
mi_tips.index

MultiIndex([(  'Male', 'Yes'),
            (  'Male',  'No'),
            ('Female', 'Yes'),
            ('Female',  'No')],
           names=['sex', 'smoker'])

Ultimately there are a ton of operations that you can do on top of this type of data. And there are equivalent multi-index operations you can do, like this:

In [ ]:
mi_tips.loc[('Male', 'No')]

tip    3.1134021
Name: (Male, No), dtype: float64

But in that way you'd have a learn a lot of details and there are always exceptions. 

So the way that I have always deal with this is simply by resetting the index.

In [ ]:
ri_tips = mi_tips.reset_index()
ri_tips

,sex,smoker,tip
0,Male,Yes,3.0511667
1,Male,No,3.1134021
2,Female,Yes,2.9315152
3,Female,No,2.7735185


Notice how we get values spread out over the full column now. So in this way it is easy to select only the male non-smokers:

In [ ]:
ri_tips[(ri_tips['smoker'] == 'No') & (ri_tips['sex'] == 'Male')]

,sex,smoker,tip
1,Male,No,3.1134021


Another way you can deal with this is to only certain indexes out:

In [ ]:
ri0_tips = mi_tips.reset_index(level=0)
ri0_tips.loc['Yes']

,sex,tip
smoker,,
Yes,Male,3.0511667
Yes,Female,2.9315152


And finally you can [pull indexes back into the index](https://github.com/knathanieltucker/pandas-tutorial/blob/master/notebooks/Indexing%20and%20Selecting.ipynb) (basically only useful for certain types of merges).

In [ ]:
ri_tips.set_index(['sex', 'smoker'])

tip
sex    smoker           
Male   Yes     3.0511667
       No      3.1134021
Female Yes     2.9315152
       No      2.7735185

In [ ]:
ri0_tips.set_index('sex', append=True)

,,tip
smoker,sex,
Yes,Male,3.0511667
No,Male,3.1134021
Yes,Female,2.9315152
No,Female,2.7735185


# Getting Single Values

The next little indexing trick is one that is mostly about speed. But it is getting and setting single values. It is a pretty simple:

In [ ]:
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


When getting/setting single values you should use the `at` function

In [ ]:
tips.at[0, 'total_bill'] = 9000
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,9000.00,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [ ]:
tips.iat[0, 0]

9000.0

If you are modifying single values of a dataframe you should always use these guys. It's faster and it is a good way to know that you are not messing up (often times modifying the data can result in odd errors).

So just to prove it's faster let's time it!

In [ ]:
%%timeit
tips.at[0, 'total_bill'] = 6

The slowest run took 15.45 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.63 µs per loop


In [ ]:
%%timeit
tips.loc['total_bill', 0] = 6

The slowest run took 17.73 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 356 µs per loop


## Where, Masks and Queries

These are things that are built into pandas that I have personally never used, mostly because they are pretty redundant and don't happen too often.

They are a bit faster, yes. But the mental space is probably not worth it. So if you wanna learn it, go for it (docs are [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#the-query-method)). If not, probably won't matter.

Let me show you how you'd duplicate mask functionality below. 

In [ ]:
df = pd.DataFrame(np.random.randn(25).reshape((5, 5)))
df.head()

,0,1,2,3,4
0,0.2613591,-2.5405017,-0.2973250,1.6764790,-0.4477959
1,0.8116989,0.6667971,-0.3014468,-1.0649429,-1.2326274
2,-0.4653553,0.1702123,0.3755633,-0.4949931,0.5230949
3,0.3802913,0.2864404,-0.1009921,-2.4431079,0.1956035
4,0.6802330,1.1413839,-0.9160501,-1.0528071,-0.3500471


In [ ]:
df.where(df > 0)

,0,1,2,3,4
0,0.2613591,NaN,NaN,1.676479,NaN
1,0.8116989,0.6667971,NaN,NaN,NaN
2,NaN,0.1702123,0.3755633,NaN,0.5230949
3,0.3802913,0.2864404,NaN,NaN,0.1956035
4,0.6802330,1.1413839,NaN,NaN,NaN


In [ ]:
df[df < 0] = np.NaN
df

,0,1,2,3,4
0,0.2613591,NaN,NaN,1.676479,NaN
1,0.8116989,0.6667971,NaN,NaN,NaN
2,NaN,0.1702123,0.3755633,NaN,0.5230949
3,0.3802913,0.2864404,NaN,NaN,0.1956035
4,0.6802330,1.1413839,NaN,NaN,NaN


## Indexing/Selecting Conclusion

So that's it. This is probably all you'll need to know about indexing/selecting. If you've got any question or comments please remember much more material is covered in our YouTube series! 

p.s. there are not really any great tutorials on this in particular, but if you know of one I should link, let me know.

## Questions?

# Pandas Row-Column Transformations

There comes a time in the life of any data scientist when he or she needs to transform the set of columns in a dataset into rows and vice versa.

This is not a common operation, but it does happen every now and then. Pandas has two set of methods to do this:

* stack and unstack
* pivot and melt

Again these sets of methods basically do the same thing.


I have found that stack and unstack are much more stable but a bit less powerful. So those are the ones I use. 

Right at the end we will go over pandas dummy variables being the last way to make the transformation. 

Check out the full documentation for both [stack and unstack](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html) and [dummy variables](http://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#computing-indicator-dummy-variables), but be warned it is a bit long :)

Okay Let's get started

In [ ]:
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


A question we might want to ask is: what is the male to female ratio on different days of the week?

To do this we might start with a groupby:

In [ ]:
tips_gb = tips.groupby(['day', 'sex']).agg({'size': 'sum'})
tips_gb

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

So we are getting somewhere, but it is a bit hard to tell the number of male and female visitors by looking at it, and you might want to do more columnwise operations comparing the male to the female visitors.

So what you might want to do is take the values in the column sex and make them into column. This is where unstacking comes in!

## Unstack

In [ ]:
tips_us = tips_gb.unstack()
tips_us

size       
sex  Male Female
day             
Thur   73     79
Fri    21     19
Sat   156     63
Sun   163     53

Notice we basically moved an index to the columns!

In [ ]:
# you could do the same with the days of the week
tips_gb.unstack(0)

size              
day    Thur Fri  Sat  Sun
sex                      
Male     73  21  156  163
Female   79  19   63   53

The problem is that now we have this odd new object as the columns:

In [ ]:
tips_us.columns

MultiIndex([('size',   'Male'),
            ('size', 'Female')],
           names=[None, 'sex'])

And while you can do things with it:

In [ ]:
tips_us[[('size', 'Male')]]

,size
sex,Male
day,
Thur,73
Fri,21
Sat,156
Sun,163


I find it a bit annoying to memorize a separate set of syntax, so I always convert it with a line of code like so (ps I wish this were in pandas core):

In [ ]:
tips_us_copy = tips_us.copy()

tips_us_copy.columns = ['__'.join(col).strip() for col in tips_us.columns.values]

In [ ]:
tips_us_copy

,size__Male,size__Female
day,,
Thur,73,79
Fri,21,19
Sat,156,63
Sun,163,53


You can of course repeat that operation as many times as you need to get the desired granularity of columns. 

But now let's try out the reverse operation. This is useful if somebody gives you data in pivot form.

## Stack

In [ ]:
tips_us.stack()

size
day  sex         
Thur Male      73
     Female    79
Fri  Male      21
     Female    19
Sat  Male     156
     Female    63
Sun  Male     163
     Female    53

Again you can unstack either column index:

In [ ]:
tips_us.stack(0)

,sex,Male,Female
day,,,
Thur,size,73,79
Fri,size,21,19
Sat,size,156,63
Sun,size,163,53


## What about Melting and Pivoting?

That is about it when it comes to stacking and unstacking. Anything you can do with melting and pivoting can be done with stacking and unstacking. Let's do a single example from pandas:

In [ ]:
cheese = pd.DataFrame({'first': ['John', 'Mary'],
                        'last': ['Doe', 'Bo'],
                        'height': [5.5, 6.0],
                        'weight': [130, 150]})
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [ ]:
# melt does stacking in one operation
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


To do this with stacking we just need to do it in two steps:

In [ ]:
cheese.set_index(['first', 'last'], inplace=True)
cheese.stack().reset_index()

,first,last,level_2,0
0,John,Doe,height,5.5
1,John,Doe,weight,130.0
2,Mary,Bo,height,6.0
3,Mary,Bo,weight,150.0


I have used melt and pivot before, but after getting a better understanding of stack and unstack I have found them more versitile and stable than the former. So why learn both!

## Dummy Variables

There is one final way to transform the values in a column into headers, and this is called making dummy vars (well not quite, if you are interested in more ways to do it you can check out my [YT video](https://www.youtube.com/watch?v=WRxHfnl-Pcs&t=2s)).

Making a dummy variable will take all the `k` distinct values in one column and make `k` columns out of them. 

Let's look at an example below:

In [ ]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
pd.get_dummies(tips.head(), columns=['sex'])

,total_bill,tip,smoker,day,time,size,sex_Male,sex_Female
0,16.99,1.01,No,Sun,Dinner,2,0,1
1,10.34,1.66,No,Sun,Dinner,3,1,0
2,21.01,3.50,No,Sun,Dinner,3,1,0
3,23.68,3.31,No,Sun,Dinner,2,1,0
4,24.59,3.61,No,Sun,Dinner,4,0,1


Notice the sex column was split into the sex_Male and sex_Female column. When the sex is female the sex_Female is 1 and 0 otherwise. And similarly for the sex_Male column.

This can be very useful for ML models and doing some types of analysis.

## Conclusion

These three ways to transform rows to columns and back again have served me quite well, and I'd be surprised if you'd need anything more than these. 

They are pretty intuitive, so you might not need to do too much practice. I actually don't know a good exercise for these guys as well - so if somebody has a good one they know of please send it over. 

# Exercises
* Source: [guipsamora](https://github.com/guipsamora/pandas_exercises/blob/master/05_Merge/Fictitous%20Names/Exercises.ipynb)
* After the Nashville Analytics Summit, check out [Fictitious Names Exercises Video Tutorial](https://youtu.be/6DbgcHBiOqo) to watch a data scientist go through the exercises.

### Introduction:

This time you will create a data again 

Special thanks to [Chris Albon](http://chrisalbon.com/) for sharing the dataset and materials.
All the credits to this exercise belongs to him.  

In order to understand about it go [here](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/).

### Step 1. Import the necessary libraries

In [ ]:
# Already completed above
# import numpy as np
# import pandas as pd

### Step 2. Create the 3 DataFrames based on the following raw data

In [ ]:
raw_data_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}

raw_data_2 = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}

raw_data_3 = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}

### Step 3. Assign each to a variable called data1, data2, data3

### Step 4. Join the two dataframes along rows and assign all_data

### Step 5. Join the two dataframes along columns and assing to all_data_col

### Step 6. Print data3

### Step 7. Merge all_data and data3 along the subject_id value

### Step 8. Merge only the data that has the same 'subject_id' on both data1 and data2

### Step 9. Merge all values in data1 and data2, with matching records from both sides where available.